In [147]:
import os

In [148]:
%pwd

'C:\\Users\\aditi\\OneDrive\\Desktop\\Chest-Cancer-ML-OPS'

In [149]:
new_directory = r"C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS"

# Change the current working directory
os.chdir(new_directory)

In [150]:
from pathlib import Path

config_file_path = Path(r"C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\config\config.yaml")
params_file_path = Path(r"C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\params.yaml")

# Check if the config file exists
if config_file_path.exists():
    print(f"{config_file_path} exists!")
else:
    print(f"{config_file_path} does NOT exist!")

# Check if the params file exists
if params_file_path.exists():
    print(f"{params_file_path} exists!")
else:
    print(f"{params_file_path} does NOT exist!")


C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\config\config.yaml exists!
C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\params.yaml exists!


In [151]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [152]:
from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import read_yaml,create_dir

In [153]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH):
        print(type(config_filepath))
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        
        create_dir([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_dir([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [154]:
import os 
import urllib.request as request
import zipfile
import gdown
from chest_cancer_classifier import logger
from chest_cancer_classifier.utils.common import get_size


In [155]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/dataingestion",exist_ok = True)
            logger.info(f"Downloading dat from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split('/')[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id , "archive(6).zip")
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_file:
            zip_file.extractall(unzip_path)

In [156]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

<class 'pathlib.WindowsPath'>
[2024-08-31 00:20:49,380 INFO : common : YAML file config\config.yaml successfully loaded]
[2024-08-31 00:20:49,382 INFO : common : YAML file params.yaml successfully loaded]
[2024-08-31 00:20:49,383 INFO : common : Created Directory at : artifacts]
[2024-08-31 00:20:49,387 INFO : common : Created Directory at : artifacts/data_ingestion]
[2024-08-31 00:20:49,389 INFO : 3709462476 : Downloading dat from https://drive.google.com/file/d/1Ri35otP5KdbnthLSfIAKRhyHUKKJjTSJ/view?usp=drive_link into file artifacts/data_ingestion/archive(6).zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1Ri35otP5KdbnthLSfIAKRhyHUKKJjTSJ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1Ri35otP5KdbnthLSfIAKRhyHUKKJjTSJ&confirm=t&uuid=9f8e9dd1-7fc4-4f2a-96b3-65cbf116a365
To: C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\archive(6).zip
100%|██████████| 124M/124M [00:25<00:00, 4.81MB/s] 


[2024-08-31 00:21:21,866 INFO : 3709462476 : Downloaded data from https://drive.google.com/file/d/1Ri35otP5KdbnthLSfIAKRhyHUKKJjTSJ/view?usp=drive_link into file artifacts/data_ingestion/archive(6).zip]
